## Prepare the inputs

- encoder_input_data: 2D array of shape `(num_images * 5, 512)`
- decoder_input_data: 3D array of shape `(num_captions, max_words_in_sentence, num_words)`
- decoder_output_data: same as decoder_input_data but offset by one timestep. decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :]

## Image Preprocessing

obtain bottleneck features

In [25]:
import numpy as np

In [15]:
# Since there are 5 captions per image, duplicate the bottleneck features
def duplicate_bottleneck_features(features):
    num_captions = 5 # 5 stands for number of captions per image
    num_rows = features.shape[0] * num_captions 

    features_dup = np.zeros((num_rows, features.shape[1]))
    for i, image in enumerate(features):
        for j in range(num_captions):
            features_dup[i*num_captions + j] = image
    return features_dup    

In [16]:
bottleneck_features = np.load('bottleneck_features/Flicker8k_bottleneck_features_VGG16_avgpooling.npz')
bottleneck_features_train = bottleneck_features["train"]
bottleneck_features_validation = bottleneck_features["validation"]
bottleneck_features_test = bottleneck_features["test"]

bottleneck_features_train_dup = duplicate_bottleneck_features(bottleneck_features_train)
bottleneck_features_validation_dup = duplicate_bottleneck_features(bottleneck_features_validation)
bottleneck_features_test_dup = duplicate_bottleneck_features(bottleneck_features_test)

In [17]:
print(bottleneck_features_train_dup.shape)
print(bottleneck_features_validation_dup.shape)
print(bottleneck_features_test_dup.shape)

(30000, 512)
(5000, 512)
(5000, 512)


## Word Embedding

In [18]:
from caption_utils import *
train_fns_list, dev_fns_list, test_fns_list = load_split_lists()

train_captions_raw, dev_captions_raw, test_captions_raw = get_caption_split()
vocab = create_vocab(train_captions_raw)
token2idx, idx2token = vocab_to_index(vocab)     
captions_data = (train_captions_raw.copy(), dev_captions_raw.copy(), test_captions_raw.copy())
train_captions, dev_captions, test_captions = process_captions(captions_data, token2idx)

In [19]:
print(len(vocab))

2531


## Caption Preprocessing


In [20]:
captions_onehot = np.load('preprocessed_captions/Flicker8k_onehot_'+str(len(vocab))+'_words.npz')
train_captions_onehot = captions_onehot["train"]
validation_captions_onehot = captions_onehot["validation"]
test_captions_onehot = captions_onehot["test"]

In [21]:
print(train_captions_onehot.shape)
print(validation_captions_onehot.shape)
print(test_captions_onehot.shape)

(30000, 37, 2531)
(5000, 37, 2531)
(5000, 37, 2531)


## Build Model

In [22]:
batch_size = 64  # Batch size for training.
epochs = 5  # Number of epochs to train for.
latent_dim = 300  # Latent dimensionality of the encoding space.
num_samples = bottleneck_features_train_dup.shape[0]

In [23]:
input_images = bottleneck_features_train_dup
target_words = train_captions_onehot

In [26]:
input_words = []
for dataset in captions_data:
    for filename in dataset:
        for caption in dataset[filename]:
            input_words.extend(caption)
            
input_words = sorted(input_words)
target_words = sorted(input_words)
num_encoder_tokens = len(vocab)
num_decoder_tokens = len(vocab)
max_encoder_seq_length = train_captions_onehot.shape[1]
max_decoder_seq_length = train_captions_onehot.shape[1]

print('Number of samples:', bottleneck_features_train_dup.shape[0])
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 30000
Number of unique input tokens: 2531
Number of unique output tokens: 2531
Max sequence length for inputs: 37
Max sequence length for outputs: 37


In [27]:
from copy import deepcopy
#input_token_index = dict(
#    [(word, i) for i, word in enumerate(input_words)])
#target_token_index = dict(
#    [(word, i) for i, word in enumerate(target_words)])

encoder_input_data = bottleneck_features_train_dup
decoder_input_data = deepcopy(train_captions_onehot)
decoder_target_data = deepcopy(train_captions_onehot)
# Change decoder_target_data: decoder_target_data is ahead of decoder_input_data by one timestep
for i, caption in enumerate(decoder_input_data):
    for j, word in enumerate(caption):
        if j > 0:
            decoder_target_data[i][j-1] = word

In [58]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(30000, 512)
(30000, 37, 2531)
(30000, 37, 2531)


(30000, 512)
(30000, 37, 2531)
(30000, 37, 2531)


-------------------------
# Experiment 

## Parameters

In [82]:
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding, BatchNormalization, RepeatVector, Concatenate, TimeDistributed
import keras.backend as K

In [73]:
emb_size = 150
lstm_size = 300
vocab_size = len(vocab)
max_length = decoder_input_data.shape[1]

In [74]:
K.clear_session()

## Image Embedding

In [75]:
image_input = Input(shape=(encoder_input_data.shape[1],))
img_emb = Dense(emb_size, activation='relu')(image_input)
img_emb = RepeatVector(1)(img_emb)
print(img_emb.shape)

(?, 1, 150)


## Word Embedding

In [79]:
caption_inputs = Input(shape=(max_length, ))
word_emb = Embedding(input_dim=vocab_size, output_dim=emb_size)(caption_inputs)
print(word_emb.shape)

(?, 37, 150)


In [81]:
seq_input = Concatenate(axis=1)([img_emb, word_emb])
print(seq_input.shape)

(?, 38, 150)


In [89]:
gru_cell = GRU(lstm_size, return_sequences=True)(seq_input)
seq_out = TimeDistributed(Dense(vocab_size, activation='softmax'))(gru_cell)
print(seq_out.shape)

(?, 38, 2531)


In [95]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(30000, 512)
(30000, 37, 2531)
(30000, 37, 2531)


In [91]:
####### Need to reshape decoder target data #######
Model(inputs=[encoder_input_data, decoder_input_data],
              outputs=decoder_target_data)

TypeError: unhashable type: 'numpy.ndarray'

# Remove lines below